## Drugs Arrested Case

### ปัจจัยที่ส่งผลต่อจำนวนคดีผู้ต้องหายาเสพติด
#### by กลุ่ม ทุกปัญหามีทางเข้าเสมอ
---

In [1]:
import pandas as pd
import numpy as np
import requests
import json, io, time, urllib3, re
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import widgets, interact
urllib3.disable_warnings()

เกริ่นเรื่องปัญหายาเสพติดในไทย

- จำนวนคดีเคส เรียงตามจังหวัด เอาปีล่าสุด - เลือกจังหวัดแสดงผลได้
- ดูเทรนด์ rank top 3 แต่ละปี
- เอาจังหวัดที่น่าสนใจ(สถิติอยู่ในท็อป3บ่อย) ex.นครศรีธรรมราช กรุงเทพ ปทุม สงขลา ขอนแก่น
- ประชากรหนาแน่น ex.กรุงเทพ ขอนแก่น
- ดูเรื่องนักท่องเที่ยว ฟีลเมืองเศรษฐกิจ ex.ชลบุรี
- หัวข้อภาคใต้ เน้นนครศรี + เรื่องกระท่อม + อาชญากรรมอื่น
- การกินเหล้า & เล่นยาสวนทางกัน

### Overview

ปัญหายาเสพติดเป็นเหมือนโรคเรื้อรังที่กัดกินอนาคตของชาติไทย มานานหลายทศวรรษ แม้จะมีความพยายามจากทุกภาคส่วนในการแก้ไข แต่ปัญหานี้ยังคงฝังรากลึกในสังคมไทย

วันนี้กลุ่มของพวกเราจะมานำเสนอข้อมูลเกี่ยวกับยาเสพติด โดยเน้นไปที่จำนวนผู้ต้องหาคดียาเสพติดในแต่ละจังหวัด และหาว่ามีปัจจัยใดบ้างที่อาจส่งผลต่อจำนวนความมากน้อยของปัญหายาเสพติดได้ รวมถึงหยิบยกบางจังหวัดที่มีข้อมูล insight ที่น่าสนใจมานำเสนอเพิ่มเติม

หมายเหตุ *ข้อมูลจำนวนผู้ต้องหาคดียาเสพติดเป็นเพียงตัวเลขหนึ่งที่สะท้อนถึงปัญหายาเสพติด ว่ามีมากน้อยเพียงใด แต่ยังไม่ใช่ตัวเลขที่เป็นตัวแทนสะท้อนข้อมูลออกมาได้ทั้งหมด 


#### จำนวนคดีผู้ต้องหาคดียาเสพติด เรียงตามจังหวัด ปี 2565

#### Top 3 Province with Highest Drugs Arrested Case 

#### ปัจจัยความหนาแน่นประชากร

#### ปัจจัยทางเศรษฐกิจ การท่องเที่ยว

#### Southern Province Insights
พวกเราอาจเคยได้ยินข่าวเรื่องการลักลอบนำเข้าขนส่งยาเสพติดกันบ่อยๆ ตามแถบชายแดนติดฝั่งพม่า หรือทางฝั่งภาคเหนือ

แต่จากข้อมูลจะพบได้ว่าจังหวัดในภาคใต้ก็มีจำนวนปัญหายาเสพติดไม่น้อยหน้าเช่นกัน

- #### จังหวัด สงขลา 


#### Reversion of Drugs & Alcohol Consumption

### ข้อเสนอแนะ